# Reinforcement Learning in Pokemon Battles
#### How well can an autonomous agent play pokemon?
---

> Smogon.com/stats maintains an index of past and present metagame data. We can use this as both a starting point, and a metric for an agent's effectiveness. Use [this documentation](https://www.smogon.com/forums/threads/official-smogon-university-usage-statistics-discussion-thread-mk-3.3591776/) for reference 

**Main Ideas**:
- Two Neural Nets are necessary:
    1. Pre-match, what pokemon do I choose?
        - Input: all pokemon and their potential moves/abilities/stats/helditems/etc. 
        - Potential for repsonsive, counter-picking?
        - Outputs 6 Pokemon, their moves, held items, IVs, Abilities, & who leads
    2. Mid-match, what action do I take for this turn?
        - Takes as input...
            - Enemy: HPs, status, types, idealized stats
            - Player: HPs, statuses, types, stats, and the same for all other Mons' 
- Both Nets will train via reinforcement learning.
- Tother, they make a competitive pokemon-playing agent that will train against itself for millenia

**Pending Steps**:
1. ~~Get the starting data~~
    - Fetch HTML, parse the .josn data
2. Create Classes both networks can leverage 
    - [Pokemon Species](https://pokeapi.co/docs/v2.html/#pokemon-section)
        - Name, Base stats, Abilities, Forms?
    - Type
        - Const Super Effectiveness Adjacency Matrix
        - Static multiplier functions?
    - [Moves](https://pokeapi.co/docs/v2.html/#moves-section)
        - Power, Accuracy, Effect(s), Type
        - Special / Physical
3. Set up framework for facilitating battle sequence
4. Prototype networks

---
### First, fetch and parse the Smogon Stats HTML
---

In [2]:
from datetime import date, timedelta

timeago = timedelta(weeks = 5)
timeframe = date.today() - timeago

timeframe = timeframe.strftime("%Y-%m")
print (timeframe)

2019-09


In [5]:
generation = 7
metagame = "ou"
weighting_baseline = 1500  # possible values: [0, 1500, 1630/1695, 1760/1825] 

INDEX = "gen" + str(generation) + metagame + "-" + str(weighting_baseline)
print (INDEX)

gen7ou-1500


In [6]:
import urllib.request, json

URL = "https://www.smogon.com/stats/" + timeframe + "/chaos/" + INDEX + ".json"
print (URL)

ret = urllib.request.urlopen(URL)
# raw_json = json.loads( ret.read().decode() )

https://www.smogon.com/stats/2019-09/chaos/gen7ou-1500.json


I'm downloading smogon/stats/[timeframe]/chaos since the JSON files allow for most open-ended analysis. It's format is as follows:

ROOT
- info {5}
    - team type
    - cutoff (weighting baseline, redundant from filename)
    - cutoff deviation (??)
    - Metagame (redundant from filename)
    - Number of battles
- data {*N Pokemon*}
    - Pokemon $i$ {10}
        - Moves ({move_name: weighted frequency I think?})
        - Checks and Counters ({ Pokemon j: [Array of len 3 w/ weighted frequencies?] })
        - Abilities ({ ability: weighted frequency? })
        - Teammates ({ Pokemon: pos/neg weighted freuency i think })
        - usage : Percentage of all teams I think
        - Items ({ item: weighted frequency? })
        - Raw Count: Integer
        - Spreads ({ (Nature:IV Distribution): Weighted frequency })
        - Happiness ({ Integer Value: Weighted frequency })
        - [Viability Ceiling](https://www.smogon.com/forums/threads/viability-ceiling-a-measure-of-how-far-a-pokemon-can-take-you.3546373/) ( [0: number of players using the mon, 1: top GXE, 2: 99th percentile GXE, 3: 95th percentile GXE] )
        
[Explaining Usage vs Raw vs Real](https://www.smogon.com/forums/threads/official-smogon-university-usage-statistics-discussion-thread-mk-3.3591776/)

### Handling battle engine computations
---

For pokemon metadata, I'm using the [Pokebase](https://github.com/PokeAPI/pokebase) python 3 wrapper for the [PokeApi](https://pokeapi.co/) database. Look at the PokeAPI URL for a JSON breakdown.

In [7]:
import pokebase as pb

# example hard-coded use case:
pikachu = pb.pokemon("pikachu")
print (pikachu.height)
print (pikachu.stats)

4
[{'base_stat': 90, 'effort': 2, 'stat': {'name': 'speed', 'url': 'https://pokeapi.co/api/v2/stat/6/'}}, {'base_stat': 50, 'effort': 0, 'stat': {'name': 'special-defense', 'url': 'https://pokeapi.co/api/v2/stat/5/'}}, {'base_stat': 50, 'effort': 0, 'stat': {'name': 'special-attack', 'url': 'https://pokeapi.co/api/v2/stat/4/'}}, {'base_stat': 40, 'effort': 0, 'stat': {'name': 'defense', 'url': 'https://pokeapi.co/api/v2/stat/3/'}}, {'base_stat': 55, 'effort': 0, 'stat': {'name': 'attack', 'url': 'https://pokeapi.co/api/v2/stat/2/'}}, {'base_stat': 35, 'effort': 0, 'stat': {'name': 'hp', 'url': 'https://pokeapi.co/api/v2/stat/1/'}}]


##### Super Effective Calculations:

In [ ]:
s=[[1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,2,1,1,.5,.5,1,1,1,1,1,1,2,1,1,.5,2],
[1,.5,1,1,0,2,.5,1,1,1,1,.5,2,1,2,1,1,1],
[1,.5,1,.5,2,1,.5,1,1,1,1,.5,1,2,1,1,1,.5],
[1,1,1,.5,1,.5,1,1,1,1,2,2,0,1,2,1,1,1],
[.5,2,.5,.5,2,1,1,1,2,.5,2,2,1,1,1,1,1,1],
[1,.5,2,1,.5,2,1,1,1,2,1,.5,1,1,1,1,1,1],
[0,0,1,.5,1,1,.5,2,1,1,1,1,1,1,1,1,2,1],
[.5,2,.5,0,2,.5,.5,1,.5,2,1,.5,1,.5,.5,.5,1,.5],
[1,1,1,1,2,2,.5,1,.5,.5,2,.5,1,1,.5,1,1,.5],
[1,1,1,1,1,1,1,1,.5,.5,.5,2,2,1,.5,1,1,1],
[1,1,2,2,.5,1,2,1,1,2,.5,.5,.5,1,2,1,1,1],
[1,1,.5,1,2,1,1,1,.5,1,1,1,.5,1,1,1,1,1],
[1,.5,1,1,1,1,2,2,1,1,1,1,1,.5,1,1,2,1],
[1,2,1,1,1,2,1,1,2,2,1,1,1,1,.5,1,1,1],
[1,1,1,1,1,1,1,1,1,.5,.5,.5,.5,1,2,2,1,2],
[1,2,1,1,1,1,2,.5,1,1,1,1,1,0,1,1,.5,2],
[1,.5,1,2,1,1,.5,1,2,1,1,1,1,1,1,0,.5,1]]
e=["Normal ","Fighting","Flying  ","Poison ",
   "Ground ","Rock   ","Bug    ","Ghost  ",
   "Steel  ","Fire   ","Water  ","Grass  ",
   "Electric","Psychic","Ice    ","Dragon ",
   "Dark   ","Fairy  "]

Consider different things to compute later: 
- [shapely values](https://www.wikiwand.com/en/Shapley_value)
- [All about ratings](https://www.smogon.com/forums/threads/everything-you-ever-wanted-to-know-about-ratings.3487422/)

References:
- [Pokemetrics](https://pokemetrics.wordpress.com/)
- [PokeAPI](https://pokeapi.co/) HUGE
- [Stats Explained](https://www.smogon.com/forums/threads/official-smogon-university-usage-statistics-discussion-thread-mk-3.3591776/)